In [9]:
import pennylane as qml
from pennylane import numpy as np
import time

In [10]:
## Include the other required notebook.
%run genBeH2geometry.ipynb

In [11]:
## Define the net charge of BeH2.
BeHHcharge = 0

In [12]:
BeH2 = qml.qchem.Molecule(BeHHsymbols,BeHHcoords,charge=BeHHcharge, basis_name ='sto-3g')
n_electrons = BeH2.n_electrons
h_vanilla, n_qubits = qml.qchem.molecular_hamiltonian(
    BeH2.symbols, BeH2.coordinates, charge=BeH2.charge, basis=BeH2.basis_name, name="BeH2_Vanillia"
)

In [15]:
hf_state = qml.qchem.hf_state(n_electrons,n_qubits)
singles, doubles = qml.qchem.excitations(n_electrons,n_qubits)

In [16]:
dev = qml.device("default.qubit",wires = h_vanilla.wires)

def circuit_ASD(params,wires):
    qml.templates.AllSinglesDoubles(params,wires,hf_state,singles=singles,doubles=doubles)

@qml.qnode(dev, diff_method='backprop')
def energy_expval_ASD(params):
    circuit_ASD(params,h_vanilla.wires)
    return qml.expval(h_vanilla)

In [17]:
def run_vqe_vanilla(energy_expval,params,opt,iterations):
    ti = time.time()
    energies = []
    runtime = []
    for i in range(iterations) :
        t1 = time.time()
        params, energy = opt.step_and_cost(energy_expval,params)
        t2 = time.time()
        runtime.append(t2-ti)
        energies.append(energy)
        if (i+1) % 5 == 0:
            print(f"Completed iteration: {i + 1}")
            print(f"Energy: {energy} Ha")
            print("Step Time:", t2-t1)
            print("----------------")
    print(f"Optimized energy: {energy} Ha")
    return energies,runtime

In [18]:
params_vanilla = np.zeros(len(doubles) + len(singles), requires_grad=True)
adam_opt = qml.AdamOptimizer(stepsize=0.02, beta1=0.9, beta2=0.99, eps=1e-08)
energies_vanilla, runtime_vanilla = run_vqe_vanilla(energy_expval_ASD,params_vanilla,adam_opt,50)

Completed iteration: 5
Energy: -15.586670350513144 Ha
Step Time: 2.758383274078369
----------------
Completed iteration: 10
Energy: -15.58869846876822 Ha
Step Time: 2.810053586959839
----------------
Completed iteration: 15
Energy: -15.590805117262656 Ha
Step Time: 2.8547332286834717
----------------
Completed iteration: 20
Energy: -15.593202569345891 Ha
Step Time: 2.8348679542541504
----------------
Completed iteration: 25
Energy: -15.593906008423554 Ha
Step Time: 2.819547414779663
----------------
Completed iteration: 30
Energy: -15.594068423789587 Ha
Step Time: 2.8880412578582764
----------------
Completed iteration: 35
Energy: -15.594489442284758 Ha
Step Time: 3.0296542644500732
----------------
Completed iteration: 40
Energy: -15.594547245830762 Ha
Step Time: 2.9240918159484863
----------------
Completed iteration: 45
Energy: -15.594646751810654 Ha
Step Time: 2.9356517791748047
----------------
Completed iteration: 50
Energy: -15.594727310862105 Ha
Step Time: 2.806852102279663
---